<a href="https://colab.research.google.com/github/Djaxis/MY-Python-Evolution/blob/main/Marmiton_Scrap_%26_Analys_Remake_2023_Brice_De_Campos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bs4
# installation du package Beautiful soup

import requests # permet de requeter les informations 
from bs4 import BeautifulSoup # Parsing - permet de recherché les informations precisément 
from urllib.request import urlopen
import pandas as pd # j'install pandas pour passer ce que je récupère du scrap en tablo, c'est plus joli

url="https://www.marmiton.org/recettes/recherche.aspx?aqt=desert-no%C3%ABl&start=84&page=8" # url adresse 
html = requests.get(url) # requete de la page 
html # REPONSE 406

from bs4 import BeautifulSoup
soup = BeautifulSoup(html.text,'html.parser')
# lecture de la page  avec tous les codes - on crée une soupe avec le code HTML
soup # affiche le code et ses balises de la page. ICI CELA PERMET D'ANALYSER LES

scrap1 = soup.find_all('div', {"class" : "recipe-card"})
# Grace à l'inspecteur d'éléments, on trouve la balise permettant d'identifier le cadre DES RECETTES
# AUTRE SYNTAX scrap1 = soup.find_all(style="border-top: 1px solid #251836; padding: 0 0 5px 7px;")
scrap1

len(scrap1)

#
titre = scrap1[0].find('h4', {'class':'recipe-card__title'}) # 
#titre.text

#note = scrap1[0].find('span', {'class':'recipe-card__rating__value'}) # 
#note 

#Avis = scrap1[0].find('span', {'class':'mrtn-font-discret'}) #
#Avis

#Ingrédients = scrap1[0].find('div', {'class':'recipe-card__description'}) # 
#Ingrédients

time = scrap1[0].find('span', {'class':'recipe-card__duration__value'})
time

urlpages = []

for i in range(2, 20):
    url = 'https://www.marmiton.org/recettes/recherche.aspx?aqt=desert-no%C3%ABl&page=' + str(i) #autre méthode
    urlpages.append(url)
urlpages

# ==== > VERSION DEFINITIVE

import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import randint
import re
import sys

# functions

def m(exp, txt):
  p = re.compile(exp)
  try:
    return re.search(p, txt).group(1)
  except:
    return "not found"   

def goodTime(time):
    originalOne = time
    time = re.sub(r"\s+", "", time)
    total = 0

    # for hours
    hour = m(r"(\d+)h", time)
    time = re.sub(r"\d+h", "", time)
    if (hour != "not found"):
        total+=int(hour)*60

    # for minuts
    mi = m(r"(\d+)min", time)
    time = re.sub(r"\d+min", "", time)
    if (mi != "not found"):
        total+=int(mi)

    # for days
    days = m(r"(\d+)j", time)
    time = re.sub(r"\d+j", "", time)
    if (days != "not found"):
        total+=int(days)*24*60

    # for leaving minuts
    if re.search("^\d+$", time):
      total+=int(time)
      time = ""

    return time, originalOne, total

def getSoup(url): # soup en boucle 
  html = requests.get(url, headers={'User-Agent': navigator})
  return BeautifulSoup(html.text, 'html.parser')

def toClean(s):
  ss = re.sub(r"\n$", "", s)
  ss = re.sub(r"\n", " ", s)
  return ss.strip()

def waitSec(maxTimeToWait):
  sleep(randint(1,maxTimeToWait))

# main

maxPage       = 5
searchList    = ['Desserts-de-Noël','Entrées-de-Noël','Poissons-de-Noël','Viandes-de-Noël']
navigator     = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'
recipes       = {}
tab           = "   "
rootUrl       = "https://www.marmiton.org"

# building a dict to contain a selection of recipes

print("****** Step 1")
for pattern in searchList: 
    print("looking at:", pattern)
    dico_chuck = {}
    start = 0
    for page in range (1,maxPage+1):
        print(tab, "processing page", page)
        waitSec(3)
        url =rootUrl+"/recettes/recherche.aspx?aqt="+pattern+"&start="+str(start)+"&page="+str(page)
        print(tab, url)
        
        soup = getSoup(url)
        answers = soup.find_all('div', {"class" : "recipe-card"}) 
        print(tab, "found", len(answers), "recipes")
        for answer in answers:

            title = answer.find('h4', {'class':'recipe-card__title'}).text

            note = answer.find('span', {'class':'recipe-card__rating__value'}).text
            note = toClean(note)

            nrate = answer.find('span', {'class':'mrtn-font-discret'}).text
            nrate = re.sub(r"sur", "", nrate)
            nrate = re.sub(r"avis", "", nrate)
            nrate = re.sub(r"\s+", "", nrate)
            
            ingre = answer.find('div', {'class':'recipe-card__description'}).text
            ingre = re.sub(r"Ingrédients : ", "", ingre)
            ingre = toClean(ingre)
            
            time = answer.find('span', {'class':'recipe-card__duration__value'}).text
            
            recipeUrl  = answer.find('a', {'class':'recipe-card-link'}).get("href") 

            if title in recipes:
              print(tab, "already known recipe", title)
            else:
              print(tab, "considering recipe", title)
              recipes[title] = {}
              recipes[title]["note"]      = note
              recipes[title]["nrate"]     = nrate
              recipes[title]["ingre"]     = ingre
              
              (timeLeft, timeOriginal, timeTotal) = goodTime(time)
              if (timeLeft == ""):
                recipes[title]["time"] = timeTotal
              else:
                print("unable to convert time, left", timeLeft, timeOriginal)
                sys.exit()

              recipes[title]["rooturl"]   = url
              recipes[title]["pattern"]   = pattern
              recipes[title]["recipeurl"] = rootUrl+recipeUrl

        start+=12

print("total is", len(recipes), "recipes found")

print("****** Step 2")

for recipe in recipes:
    recipeUrl = recipes[recipe]["recipeurl"]
    print("looking at", recipe, recipeUrl)
    soup = getSoup(recipeUrl)
    waitSec(3)
    div = soup.find('div', {'class':'recipe-infos__budget'})
    recipes[recipe]["budget"] = div.span.text
    div = soup.find('div', {'class':'recipe-infos__level'})
    recipes[recipe]["level"] = div.span.text
    div = soup.find('div', {'class':'recipe-infos-users__notebook'})
    
    # handling likes and also non-numeric ones eg 1.9k
    likes = div.span.text
    if re.search("\d+\.\dk", likes) or re.search("\d+k", likes):
        likes = re.sub(r"\.", "", likes)
        likes = re.sub(r"k", "", likes)
        likes = int(likes) * 100
    
    if re.search("^\d*$", str(likes)):
        recipes[recipe]["likes"] = likes
    else:
        print("non-numeric likes:", likes)
        sys.exit()

print("bye.")

In [ ]:
df = pd.DataFrame.from_dict(data=recipes, orient='index')
df



In [ ]:
# sandbox
import sys

def m(exp, txt):
  p = re.compile(exp)
  try:
    return re.search(p, txt).group(1)
  except:
    return "not found"   

def goodTime(time):
    time = re.sub(r"\s+", "", time)
    total = 0

    # for hours
    hour = m(r"(\d+)h", time)
    time = re.sub(r"\d+h", "", time)
    if (hour == "not found"):
        print("hour time conversion failed")
        sys.exit()
    else:
        total+=int(hour)*60

    # for minuts
    mi = m(r"(\d+)min", time)
    time = re.sub(r"\d+min", "", time)
    if (mi == "not found"):
        print("min time conversion failed")
        sys.exit()
    else:
        total+=int(mi)

    # for days
    days = m(r"(\d+)j", time)
    time = re.sub(r"\d+j", "", time)
    if (days == "not found"):
        print("days time conversion failed")
        sys.exit()
    else:
        total+=int(days)*24*60

    return time, total

print(goodTime("1j 2h10min"))

### SENTIMENTS ANALYSIS

In [ ]:

import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import seaborn as sns
import matplotlib.pyplot as plt
import requests # permet de requeter les informations 
from bs4 import BeautifulSoup # Parsing - permet de recherché les informations precisément 
from urllib.request import urlopen
import pandas as pd


csvFile = "https://raw.githubusercontent.com/cyrilbs/wild-repo/main/marmitton.csv"
df = pd.read_csv(csvFile)
df

In [ ]:
df['ingre'].isnull().values.any()
df['ingre'].isnull().sum()

In [ ]:
df['ingre'].isnull().values.any()
df['budget'].isnull().sum()

In [ ]:
df['budget'] = df['budget'].fillna('Coût moyen')

In [ ]:
df['Unnamed'].isnull().sum()
print(df['Unnamed'])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer()
#vectorizer = TfidfVectorizer()

X = df["ingre"]
y = df["budget"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=35, train_size = 0.75)

vectorizer.fit(X_train)

X_train_CV = vectorizer.transform(X_train)
X_test_CV = vectorizer.transform(X_test)

model = LogisticRegression(max_iter=150).fit(X_train_CV,y_train)

print(model.score(X_train_CV,y_train))
print(model.score(X_test_CV,y_test))